# Module 4: Evaluation and Monitoring
* https://github.com/DataTalksClub/llm-zoomcamp/tree/main/04-monitoring

## Introduction to monitoring answer quality
* https://youtu.be/OWqinqemCmk?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R
* https://github.com/DataTalksClub/llm-zoomcamp/tree/main/04-monitoring#41-introduction-to-monitoring-answer-quality

## Offline vs Online (RAG) evaluation
* https://youtu.be/yTKGSqkhgI4?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R
* https://github.com/DataTalksClub/llm-zoomcamp/tree/main/04-monitoring#42-offline-vs-online-rag-evaluation

## Generating data for offline RAG evaluation
* https://youtu.be/yTO5sRw6x78?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R
* https://github.com/DataTalksClub/llm-zoomcamp/tree/main/04-monitoring#43-generating-data-for-offline-rag-evaluation
* https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/offline-rag-evaluation.ipynb

In [13]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 10.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.9 MB/s eta 0:00:00


In [14]:
# Import libraries
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [2]:
# Load documents with IDs
# https://youtu.be/yTO5sRw6x78?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=85
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
documents[10]

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
 'section': 'General course-related questions',
 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
 'course': 'data-engineering-zoomcamp',
 'id': 'ea739c65'}

In [5]:
# Load ground truth
# https://youtu.be/yTO5sRw6x78?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=105
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [6]:
ground_truth[10]

{'question': 'Are sessions recorded if I miss one?',
 'course': 'machine-learning-zoomcamp',
 'document': '5170565b'}

In [7]:
doc_idx = {d['id']: d for d in documents}
doc_idx['5170565b']['text']

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

In [11]:
# Index data
# https://youtu.be/yTO5sRw6x78?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=151
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [15]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x71e053e69cc0>: Failed to establish a new connection: [Errno 111] Connection refused))